In [9]:
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
%dotenv

In [21]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text):
    embedding = client.embeddings.create(
      model="text-embedding-ada-002",
      input=text,
      encoding_format="float"
    )
    return embedding.data[0].embedding

In [29]:
embedding = get_embedding("Павло Тичина")

In [31]:
import pymongo
import os
from pprint import pprint

mongo_uri = os.environ["MONGODB_URI"]

connection = pymongo.MongoClient(mongo_uri)
database = 'prt-search'
collection = 'test-embeddings'

def vector_search(embedding):
    pipeline = [
        {
            "$vectorSearch": {
                "queryVector": embedding,
                "path": "text_embedding",
                "numCandidates": 5,
                "limit": 2,
                "index": "text_embeddings",
            }
        },
        {
            "$project":{
                "text_embedding":0,
                "_id":0,
                'score': {
                    '$meta': 'searchScore'
                }
            }
        }
    ]
    
    return list(connection[database][collection].aggregate(pipeline))

pprint(vector_search(embedding))

[{'text': 'О панно Інно, панно Інно'}, {'text': 'Садок вишневий коло хати'}]


In [32]:
embedding = get_embedding("Тарас Шевченко")
pprint(vector_search(embedding))

[{'text': 'О панно Інно, панно Інно'}, {'text': 'Садок вишневий коло хати'}]


In [33]:
embedding = get_embedding("Хрущі над вишнями")
pprint(vector_search(embedding))

[{'text': 'Садок вишневий коло хати'}, {'text': 'О панно Інно, панно Інно'}]


In [34]:
embedding = get_embedding("Павло Тичина")
pprint(vector_search(embedding))

[{'text': 'О панно Інно, панно Інно'}, {'text': 'Садок вишневий коло хати'}]


In [35]:
embedding = get_embedding("Armstrong")
pprint(vector_search(embedding))

[{'text': 'What a wonderful life'}, {'text': 'Садок вишневий коло хати'}]
